<a href="https://colab.research.google.com/github/satish860/see_food/blob/main/MNIST_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torchvision import datasets,transforms

In [3]:
train_data = datasets.MNIST('/content/drive/MyDrive/MNIST',download=True,train=True,transform=transforms.ToTensor())

In [4]:
test_data = datasets.MNIST('/content/drive/MyDrive/MNIST',download=True,train=False,transform=transforms.ToTensor())

In [6]:
from torch.utils.data import DataLoader,random_split

In [7]:
train_ds, val_ds = random_split(train_data,[50000,10000])

In [8]:
batch_size = 64
train_dl = DataLoader(train_ds,batch_size=batch_size,shuffle=True)
val_dl = DataLoader(val_ds,batch_size=batch_size)

# Define Device to switch between CPU to GPU

In [9]:
def get_default_device():
  if torch.cuda.is_available():
    return torch.device('cuda')
  return torch.device('cpu')

In [10]:
get_default_device()

device(type='cuda')

In [12]:
def to_device(data,device):
  if isinstance(data, (list,tuple)):
    return [to_device(x, device) for x in data]
  return data.to(device)

In [13]:
class DeviceDataLoader():
  def __init__(self,dl,device):
    self.dl = dl
    self.device = device

  def __iter__(self):
    for xb in self.dl:
      yield to_device(xb,self.device)
    
  def __len__(self):
    return len(self.dl)

# Define DataLoader

In [14]:
train_dl = DeviceDataLoader(train_dl,get_default_device())
val_dl = DeviceDataLoader(val_dl,get_default_device())

# Define Linear Model for MNIST

In [15]:
import torch.nn as NN

In [16]:
input_size = 784 # 28 * 28 * 1 - Image size of MNIST
num_of_classes = 10

In [36]:
class MNISTLinear(NN.Module):
  def __init__(self):
    super().__init__()
    self.linear= NN.Linear(input_size,num_of_classes)
    # self.linear2 = NN.Linear(32,num_of_classes)
  
  def forward(self,xb):
    xb = xb.view(xb.size(0),-1)
    return self.linear(xb)

In [37]:
model_test = to_device(MNISTLinear(),get_default_device())

In [38]:
import torch.nn.functional as F

In [39]:
for x,y in train_dl:
  preds = model_test(x)
  probs =F.softmax(preds,dim=1)
  _,pred_label = torch.max(probs,dim=1)
  print(pred_label)
  print(y)
  break;

tensor([7, 5, 3, 1, 6, 3, 3, 7, 3, 3, 0, 3, 3, 0, 0, 3, 3, 3, 0, 3, 0, 3, 7, 3,
        0, 7, 0, 0, 7, 9, 3, 3, 3, 0, 6, 3, 7, 3, 7, 6, 3, 3, 7, 7, 3, 3, 3, 3,
        0, 3, 7, 7, 3, 3, 0, 0, 7, 7, 0, 7, 3, 3, 3, 0], device='cuda:0')
tensor([1, 1, 8, 7, 9, 2, 3, 1, 6, 0, 4, 6, 0, 8, 3, 6, 6, 5, 6, 6, 2, 6, 1, 0,
        8, 6, 3, 5, 1, 3, 5, 2, 6, 2, 3, 3, 9, 6, 1, 4, 5, 0, 4, 1, 3, 6, 9, 6,
        8, 5, 9, 1, 4, 5, 3, 4, 5, 1, 4, 7, 8, 2, 5, 3], device='cuda:0')


# Define Accuracy

In [25]:
def accuracy(preds,label):
  _,pred_label = torch.max(preds,dim=1)
  return torch.tensor(torch.sum(pred_label == label).item()/len(label))

In [26]:
accuracy(preds,y)

tensor(0.2031)

In [27]:
loss_fn = F.cross_entropy

In [28]:
import torch.optim as Optim

In [40]:
def fit(epochs,lr,train_dl,val_dl,model,loss_fn,optim_fn=Optim.SGD):
  optimizer = optim_fn(model.parameters(),lr)

  for epoch in range(epochs):
    train_loss = []
    train_acc = []
    for xb,y in train_dl:
      preds = model(xb)
      loss = loss_fn(preds,y)
      train_loss.append(loss)
      acc = accuracy(preds,y)
      train_acc.append(acc)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    avg_loss = torch.stack(train_loss).mean()
    avg_acc = torch.stack(train_acc).mean()
    print("Epoch [{}], train_loss: {:.4f}, train_acc: {:.4f}".format(epoch, avg_loss,avg_acc))

In [41]:
mnist_model = to_device(MNISTLinear(),device=get_default_device())
fit(10,lr=0.001,train_dl=train_dl,val_dl=val_dl,model=mnist_model,loss_fn=loss_fn)

Epoch [0], train_loss: 1.9514, train_acc: 0.5464
Epoch [1], train_loss: 1.4778, train_acc: 0.7673
Epoch [2], train_loss: 1.2085, train_acc: 0.7988
Epoch [3], train_loss: 1.0434, train_acc: 0.8157
Epoch [4], train_loss: 0.9338, train_acc: 0.8246
Epoch [5], train_loss: 0.8557, train_acc: 0.8328
Epoch [6], train_loss: 0.7971, train_acc: 0.8387
Epoch [7], train_loss: 0.7515, train_acc: 0.8433
Epoch [8], train_loss: 0.7144, train_acc: 0.8477
Epoch [9], train_loss: 0.6840, train_acc: 0.8513


In [42]:
class MNISTLinear2(NN.Module):
  def __init__(self):
    super().__init__()
    self.linear= NN.Linear(input_size,32)
    self.linear2 = NN.Linear(32,num_of_classes)
  
  def forward(self,xb):
    xb = xb.view(xb.size(0),-1)
    out = self.linear(xb)
    out = F.relu(out)
    out = self.linear2(out)
    return out

In [44]:
mnist_model = to_device(MNISTLinear2(),device=get_default_device())
fit(10,lr=0.01,train_dl=train_dl,val_dl=val_dl,model=mnist_model,loss_fn=loss_fn)

Epoch [0], train_loss: 1.4147, train_acc: 0.6702
Epoch [1], train_loss: 0.5767, train_acc: 0.8600
Epoch [2], train_loss: 0.4353, train_acc: 0.8844
Epoch [3], train_loss: 0.3845, train_acc: 0.8947
Epoch [4], train_loss: 0.3571, train_acc: 0.9002
Epoch [5], train_loss: 0.3394, train_acc: 0.9048
Epoch [6], train_loss: 0.3265, train_acc: 0.9076
Epoch [7], train_loss: 0.3159, train_acc: 0.9109
Epoch [8], train_loss: 0.3071, train_acc: 0.9128
Epoch [9], train_loss: 0.2991, train_acc: 0.9147


# Test against Validation Set

In [ ]:
def evalute(model,)